# GPU-Accelerated Numerical Computing with MatX

## Tutorial List
1. [Introduction](01_introduction.ipynb)
2. [Operators](02_operators.ipynb)
3. [Executors](03_executors.ipynb)
4. Radar Pipeline Example] (this tutorial)

## Basic Radar Pipeline
This lesson uses several of the MatX features from previous lessons to show an entire radar signal processing pipeline inside of a single class. The processing is broken down into four parts:

- Pulse compression
- Pulse canceller
- Doppler
- CFAR detection

These functions each run serially in the same stream and use the output from the previous step as input. The stream is an input to the class constructor, so multiple radar pipelines can be instantiated and run in parallel in different streams. 

Each pass of this processing chain works across a set of input pulses, and the pipeline would continuously run as more pulses arrive. A full example of this radar pipeline can be found in the MatX source code under examples/simple_radar_pipeline.cu.

### Setup
#### Member Variables
Several data tensor variables are configured to set parameters for the entire processing chain. In production code, we may want to save GPU memory by reusing memory for different stages, but in this example we explicitly allocate memory in each step for clarity. This training also combines the example into a single C++ class. This allows a more composable code structure by providing automatic deallocations when an object is freed along with the ability to easily create multiple radar pipelines by simply instantiating more objects. In previous examples, the default CUDA stream was used since we were demonstrating basic functionality. In a real application, all work is done in a non-default stream to allow more parallel and controlled workloads. The `BasicRadar` object takes an input CUDA stream that it uses in each of the kernel launches so multiple instance can be completely isolated. The class also provides accessor functions to get to the internal data members, which is useful for having test points throughout the pipeline. Typically only the inputs and outputs of the entire pipeline are needed, but having access to intermediate points allows for testing functions separately. These accessors are also used inside of each example's `main()` function for setting random data on the inputs to provide more meaningful calculations.

The radar pipeline has several configurable variables for controlling the number of pulses, channels, samples, and waveform length. For this tutorial we fix the values at compile time, but these can be modified or loaded from a config file for more flexibility.

As in previous examples, we first create of all our `tensor_t` objects, which will perform managed memory allocations with the default constructor:

```c++
  tensor_t<float,3> *normT = nullptr;
  tensor_t<float,3> *ba = nullptr;
  tensor_t<int,3> *dets = nullptr;
  tensor_t<float, 1> *cancelMask = nullptr;
  tensor_t<float,3> *xPow = nullptr;
  tensor_t<complex, 1> *waveformView = nullptr;
  tensor_t<float, 0> *norms = nullptr;
  tensor_t<complex,3> *inputView = nullptr;
  tensor_t<complex,3> *tpcView = nullptr;
  tensor_t<float, 2> *cfarMaskView = nullptr;
```

As shown above, we have every size tensor from a 0-rank up to a rank 3 tensor, and even use different types for different scenarios. We allocate all the `tensor_t` variables as pointers to keep the complex initialization outside of the member list. Since the variables are just pointers, no memory is allocated at this point.

#### Constructor
The RadarPipeline constructor calls operator `new` to construct each of the view objects above, and allocate all the backing managed memory. The constructor allocation uses similar syntax to the variable declaration, but a size is also specified using an initializer-list-like syntax:

```c++
...
inputView       = new tensor_t<complex,3>({numChannels, numPulses, numSamplesRnd});
...
```

The radar pipeline uses several static tensors for each pass that only need to be initialized once. The first tensor we configure is to set the filter coefficients of our cancellation mask in the pulse canceller below. This is a small, 3-element 1D tensor implementing a high-pass filter:

```c++
  cancelMask = {1, -2, 1};  
```  

The CFAR mask is a 2D tensor representing which cells to search for signal and noise. The cell where the target is expected is considered to be the power of the signal, while the reference cells, or surrounding cells, are used as a noise estimate. There are also guard cells used around the signal as cells that are not used for signal or noise. 

The CFAR mask is convolved with a 3D tensor of ones of the same size to get a weighted estimate of the mask:

```c++
  tensorShape_t s({numChannels, numPulsesRnd, numCompressedSamples});
  conv2d(*normT, ones(s), *cfarMaskView, matxConvCorrMode_t::MATX_C_MODE_FULL, stream);      
```

The first line creates a new `tensorShape_t` object that will be used to create our generator. Generators were introduced in example 3 as a way to generate data on-the-fly without storing it in memory. In this case, we create a 3D `ones` generator that will emit the value `1` for all valid indices. The second line uses the `ones` generator and the previously-created CFAR mask to convolve together into the output view `normT`. You may have noticed that the CFAR mask and `ones` generator are not the same rank, which usually doesn't make sense for a 2D convolution. In this case, MatX is using a *broadcast* feature where it uses the lower rank tensor to repeat across the higher ranked tensor. In other words, `cfarMask` is treated as a *cloned* tensor where the highest rank would be the number of channels during the convolution, and the output is a tensor matching the size and rank of the higher-ranked input tensor. We can print `normT` to verify:

```c++
  auto rv = radar.GetNormT().Slice<1>({0, 0, 0}, {matxSliceDim, matxSliceDim, 16});
  print(rv);
```


In [ ]:
!./exercises/compile_and_run.sh example4_init

The last part of the constructor performs prefetching of all data objects. This ensures that our data is paged into GPU memory and not just resident on the CPU. Without performing this step, the first pass of the pipeline will run slower due to the page faults.

## Pulse Compression
Pulse compression is a technique used in radar signal processing for improving the range resolution and for determining how close two targets are together. Typically long pulses are needed to capture enough energy back from the target, but longer pulses are worse for differentiating target range. Pulse compression modulates a long pulse in either time or phase to increase the range.

Before doing any signal processing, we need to get views for all the data objects we use:

```c++
  auto waveformPart = waveformView->Slice({0}, {waveformLength});  
  auto waveformT = waveformView->Clone<3>({numChannels, numPulses, matxKeepDim});  
  auto waveformFull = waveformView->Slice({0}, {numSamplesRnd}); 
  auto x = *inputView;
```

All of these types of views we have seen in previous exercises, but just to reiterate, two of the views are slicing off a subset of the entire data, one is a view of all the data, and one is cloning a 1D tensor (`waveformView`) to a 3D tensor by using `Clone`.

The first stage in pulse compression is to apply a window function to the input waveform:
```c++
  auto op = set(waveformPart, waveformPart * hamming_x(s));
  exec(op, stream); 
```

In this case we're applying a Hamming window to our partial waveform view. `hamming` is a generator function that generates Hamming window values at each point defined in the tensor. Next, we compute the L2 norm of the partial waveform. The L2 norm is done in two steps currently: an I^2 + Q^2 reduction, followed by a square root on the output of the reduction:

```c++
  sum(norms, norm(waveformPart), stream);
  exec(norms, sqrt(norms), stream);
```

With the L2 norm calculated, each sample from the waveform vector is scaled by this norm by dividing element-wise:

```c++
exec(waveformPart, waveformPart / norms, stream);  
```

Next, we take an FFT of the waveform, followed by the complex conjugate:

```c++
  fft(waveformFull, waveformPart, waveformFull.Size(), stream);
  exec(waveformFull, conj(waveformFull), stream);
```            

The FFT above uses a different input tensor size (`waveformPart`) that's smaller than the output tensor (`waveformFull`). When MatX sees this signature, it will zero-pad the input tensor prior to taking the FFT. 

With the waveform now in the frequency domain, the input signal is converted to the frequency domain by `fft`. Next, an element-wise multiply is performed on the input signal and waveform, followed by an IFFT. This uses the convolution theorem to perform a convolution in the frequency domain before converting back to the time domain. This step is also routinely referred to as matched filtering, where one is trying to receive the signal that was transmitted at an earlier time:

```c++
  fft(x, x, stream);
  exec(x, x * waveformT, stream);
  ifft(x, x, stream);
```            

Since the cuFFT library does not normalize the signal by 1/N as MATLAB and Python do, we add an additional normalize step to scale the signal by the fastest-changing dimension:

```c++
exec(x, x * (1.0f/x.Size(2)), stream);
```

Finally, we print a portion of the signal to show the final pulse compression results.

In [ ]:
!./exercises/compile_and_run.sh example4_pc

## Pulse Canceller
A pulse canceller filters a signal to help differentiate between background noise and stationary objects (like mountains) using a high-pass filter. The filter taps have been pre-defined inside the constructor section above. The filtering is implemented with a time domain convolution, which uses the `conv1d` function on the input signal. Before the convolution happens, two sliced and permuted views are created:

```c++
  auto x = inputView->Permute({0,2,1}).Slice({0, 0, 0}, {numChannels, numCompressedSamples, numPulses});
  auto xo = tpcView->Permute({0,2,1}).Slice({0, 0, 0}, {numChannels, numCompressedSamples, numPulses});
```              

The `Permute` function is rearranging the axes such that the convolution is performed on the fastest-changing dimension (`numPulses`), and the `Slice` function is ensuring that we only looking at the compressed sample count in the second dimension. Again, this does not do anything to the data on the host or device; it simply changes which elements are pulled out when accessing the tensors with `operator()`. With the views created, the convolution is performed:

```c++
   conv1d(xo, x, cancelMask, matxConvCorrMode_t::MATX_C_MODE_SAME, stream);  
```

We use convolution mode `MATX_C_MODE_SAME` to keep the input and output signals the same size, effectively cutting off the filter ramp-up and ramp-down time. Viewing a portion of each signal shows the convolution inputs and outputs:

In [ ]:
!./exercises/compile_and_run.sh example4_tpc

## Doppler Processing
The Doppler processing step converts the range-pulse data to range-Doppler data by taking an FFT in the Doppler (pulses) dimension. Doppler processing is often use to differentiate slow moving objects (birds, hot air balloons, etc) from fast moving objects (like planes and rockets). Before performing the FFT, a Hamming window is applied to each pulse by using the `hamming_y` function:

```c++
  auto xc = tpcView->Slice({0, 0, 0}, {numChannels, cpulses, numCompressedSamples});
  auto xf = tpcView->Permute({0,2,1});

  // Shape of hamming window
  tensorShape_t<3> s({numChannels, numPulses - (cancelMaskData->Size(0) - 1), numCompressedSamples});

  exec(xc, xc * hamming_y(s), stream);
  fft(xf, xf, stream);  
```

Note that the FFT takes a permuted view as input. When possible, MatX will detect the permuted view and pass this into the underlying FFT library as a large stride. This can be inefficient in certain cases, and it may make sense to call `transpose` prior to the FFT to transpose the data in memory before calling the FFT.

In [ ]:
!./exercises/compile_and_run.sh example4_doppler -lcufft

## CFAR Detection
The last step in the pipeline is the constant false alarm rate (CFAR) detection. CFAR detection is broadly used to filter observible signals from noise by setting a threshold for observation. A filter mask was created in the constructor to represent the "field of view" that we are looking for a target in. By describing the field of view, we can differentiate what parts of the signal we believe are signal power and noise power. 

CFAR detection begins by taking the signal power of the last stage by summing the squares of all complex numbers (I^2 + Q^2). This is done by using the MatX `norm` operator:

```c++
exec(xdPow, norm(cfarIn), stream);
```

xdPow now contains the sum of the squares of each element. Using the computed power per cell, we apply the CFAR mask that was computed in the constructor. The mask is applied using a 2D convolution from the MatX `conv2d` function:

```c++
  conv2d(ba, xdPow, cfarMask, matxConvCorrMode_t::MATX_C_MODE_FULL, stream);
```

To ignore edge elements of the signal, a view of the mask is created by ignoring the first and last halves of the filter length on the mask. This same view size is used to use the normalize all convolution outputs by our normalized mask that was precomputed in the constructor:

```c++
  auto normTrim = normT->Slice(
      {0, cfarMaskY / 2, cfarMaskX / 2}, 
      {numChannels, numPulsesRnd + cfarMaskY / 2, 
      numCompressedSamples +  cfarMaskX / 2}); 
                  
  auto baTrim = ba->Slice(
      {0, cfarMaskY / 2, cfarMaskX / 2}, 
      {numChannels, numPulsesRnd + cfarMaskY / 2, 
      numCompressedSamples + cfarMaskX / 2}); 

  exec(baTrim, baTrim / normTrim, stream);
```  

With the background averages computed, the final step in CFAR detection is to compare all cell powers to a threshold. The threshold we use is a combination of the cell power tensor, the background averages, and our mask norm initialized in the constructor. All of these operations can be done using a combination of several `exec` statements until the final detection tensor is computed. However, all of these operations are simply a large, independent calculation that can be done without any synchronization between threads. To accomplish this, we define a custom operator named `calcDets` that takes the three tensors and a probability of false alarm threshold as inputs, and outputs a single detection tensor. Removing some boilerplate code for clarity, the operator is defined as follows:

```c++
template<class O, class I1, class I2, class I3, class I4> 
class calcDets {
  private: 
    O out_;
    I1 xpow_;
    I2 ba_;
    I3 norm_;
    I4 pfa_;

  public:
    calcDets(O out, I1 xpow, I2 ba, I3 norm, I4 pfa) : 
      out_(out), xpow_(xpow), ba_(ba), norm_(norm), pfa_(pfa) {}

    __device__ inline void operator()(index_t idz, index_t idy, index_t idx) { 
      typename I1::type xpow = xpow_(idz, idy, idx);
      typename I2::type ba = ba_(idz, idy, idx);
      typename I2::type norm = norm_(idz, idy, idx);
      typename I2::type alpha = norm * (std::pow(pfa_, -1.0/norm) - 1);
      out_(idz, idy, idx) = (xpow > alpha * ba) ? 1 : 0;
    }
};
```

The `calcDets` operator uses the required operator signature of a constructor taking a single output tensor, followed by a list of input parameters. When the operator is indexed, each thread will perform the computations in `operator()` and computed an their own `alpha` value before outputting either a 1 or a 0 depending on if the detection threshold was passed. The operator is called like all other operators:

```c++
matxExecuteOp(calcDets(dets, xdPow, baTrim, normTrim, pfa), stream);
```

In [ ]:
!./exercises/compile_and_run.sh example4_cfar -lcufft

## Conclusion
In this tutorial we showed how to use MatX to implement an entire radar processing pipeline. Using a class structure, we created code that was composable and easy to start as many pipelines in parallel as we need. This could extend to multiple devices on the same stream or even multiple GPUs with only a few more lines of code. We also introduced a custom operator, convolution functions, and several ways to use a `View` when working with these transformations. By looking over the `simple_radar_pipeline.h` example in the MatX source, you should be equipped with the knowledge needed to create your own application in MatX. For more examples please see the documentation or examples. 